In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import cPickle as pickle
from datetime import datetime
from scipy.optimize import curve_fit
#pd.set_option('notebook_repr_html', False)
#pd.set_option('display.max_rows', 20)

from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline, FeatureUnion

import seaborn as sns
import matplotlib
matplotlib.rcParams['savefig.dpi'] = 1.5 * matplotlib.rcParams['savefig.dpi']
from matplotlib import pyplot as plt
from collections import OrderedDict
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models.glyphs import Circle
from bokeh.models import (
    GMapPlot, GMapOptions, Range1d, ColumnDataSource,
    PanTool, WheelZoomTool, BoxZoomTool, BoxSelectTool, ResetTool, PreviewSaveTool, HoverTool)

In [ ]:
trips_df = pickle.load(open('trips_2014.pickle', 'r'))
stations_df = pickle.load(open('stations_2014.pickle', 'r'))
gaps_by_month_bikeid = pickle.load(open('gaps_by_month_bikeid.pickle', 'r'))

months = range(1, 13)
bikeids = np.sort(trips_df.bikeid.unique())
gaps_by_bikeid = {bikeid: np.concatenate([gaps_by_month_bikeid[month][bikeid] for month in months]) for bikeid in bikeids}
gaps_by_month = {month: np.concatenate([gaps_by_month_bikeid[month][bikeid] for bikeid in bikeids]) for month in months}
all_gaps = np.concatenate([gaps_by_month[month] for month in months])  # the only useful gaps data

print len(trips_df), len(all_gaps), len(stations_df)

In [ ]:
def func(t, tau, A):
    return A*np.exp(-t/tau)

short_gaps = all_gaps[(all_gaps>0.5) & (all_gaps<4.5)]
hist_data = np.histogram(short_gaps, bins=16)
xs = (hist_data[1][:-1]+hist_data[1][1:])/2
ys = hist_data[0]
tau, A = curve_fit(func, xs, ys, p0=[2.0, 10**5])[0]
print 'average time between two trips is %.2f hours' % tau

plt.suptitle('Histogram of Time Intervals', fontsize=18)

plt.subplot(2,1,1)
plt.hist(short_gaps, bins=16, alpha=0.5, color='b')
xs0 = np.linspace(0.5, 4.5, 25)
plt.plot(xs0, func(xs0, tau, A),'r--')
plt.xlim(0.5, 4.5)
plt.ylabel('Count', fontsize=16)

plt.subplot(2,1,2)
long_gaps = all_gaps[(all_gaps>4) & (all_gaps<12)]
plt.hist(long_gaps, bins=16, color='b', alpha=0.5)
plt.xlim(4, 12)
plt.xlabel('Hour', fontsize=16)
plt.ylabel('Count', fontsize=16)
plt.show()

In [ ]:
# label: number of maintenances

cutoff = 48.0  # 5*tau+8.5 = 16!
n_maints = np.array([len(filter(lambda t: t>cutoff, gaps_by_bikeid[bikeid])) for bikeid in bikeids])
plt.suptitle('Histogram of Maintenances', fontsize=18)
plt.hist(n_maints, bins=16, color='b', alpha=0.5)
plt.xlabel('Number', fontsize=16)
plt.ylabel('Count', fontsize=16)
plt.show()

In [ ]:
n_trips = pickle.load(open('n_trips_by_bikeid.pickle', 'r'))
plt.hist(n_trips, bins=15, color='b', alpha=0.5)
plt.xlabel('Number of trips')
plt.ylabel('Count')
plt.show()

In [ ]:
#n_trips_seasons = pickle.load(open('n_trips_seasons.pickle', 'r'))
n_trips_months = pickle.load(open('n_trips_months.pickle', 'r'))

index = np.arange(12)
bar_width = 0.75
plt.bar(index, n_trips_months.sum(axis=0), bar_width, color='b', alpha=0.5)
plt.title('Number of Trips in a Year')
plt.ylabel('Count')
plt.xticks(index+bar_width/2, ('JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'SEP', 'AUG', 'OCT', 'NOV', 'DEC'))
#plt.xticks(index+bar_width/2, ('Winter', 'Spring', 'Summer', 'Autum'))
plt.show()

In [ ]:
# third feaures: number of pass-bys at n_clusters of bike stations

n_clusters = 9  # optimized from a smaller sample
kmeans = KMeans(n_clusters=n_clusters).fit(stations_df[['latitude','longitude']].values)
stations_df['cluster'] = kmeans.labels_
lons = stations_df.longitude
lats = stations_df.latitude
names = stations_df.name
# colors used to maximally distinguish clusters
color_map_1 = ['#d53e4f', '#f46d43', '#fdae61', '#fee08b', '#ffffbf', '#e6f598', '#abdda4', '#66c2a5', '#3288bd']
# colors used to show the graduate change among clusters
color_map_2 = ['#fff5f0', '#fee0d2', '#fcbba1', '#fc9272', '#fb6a4a', '#ef3b2c', '#cb181d', '#a50f15', '#67000d']
colors = [color_map_1[c] for c in stations_df.cluster]

In [ ]:
output_notebook()
map_options = GMapOptions(lat=41.8827, lng=-87.6227, map_type="roadmap", zoom=11)
plot = GMapPlot(x_range=Range1d(), y_range=Range1d(), map_options=map_options, title="Clustering of Bike Stations")
plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(), ResetTool(), PreviewSaveTool(), HoverTool())
source = ColumnDataSource(data=dict(x = lons, y = lats, radius = [12.5]*300, color = colors, name = names))
circle = Circle(x="x", y="y", size="radius", fill_color="color", line_color=None, fill_alpha=0.75)
plot.add_glyph(source, circle)
hover = plot.select(dict(type=HoverTool))
hover.point_policy = "follow_mouse"
hover.tooltips = OrderedDict([("Station", "@name")])
show(plot)

In [ ]:
n_trips_clusters = pickle.load(open('n_trips_clusters.pickle', 'r'))

index = np.arange(n_clusters)
bar_width = 0.75
plt.bar(index, n_trips_clusters.sum(axis=0), bar_width, color=color_map, alpha=0.5)
plt.title('Number of Pass-bys in %d Clusters of Bike Stations' % n_clusters)
plt.ylabel('Count')
plt.xticks(index+bar_width/2, ['CL %s' % i for i in index])
plt.show()

In [ ]:
# Month Model

np.random.seed(42)
random_indices = np.random.permutation(len(bikeids))
X_random = n_trips_months[random_indices]
y_random = n_maints[random_indices]
X_train, X_test, y_train, y_test = train_test_split(X_random, y_random, test_size=0.2, random_state=42)

estimator = LinearRegression()
estimator.fit(X_train, y_train)
print estimator.score(X_test, y_test)

#parameters = {'n_estimators': range(30, 50, 1)}
#estimator = GridSearchCV(RandomForestRegressor(), parameters, cv=5)
estimator = RandomForestRegressor(n_estimators=79)
estimator.fit(X_train, y_train)
print estimator.score(X_test, y_test)
#print estimator.best_params_

In [ ]:
# Station-Cluster Model

np.random.seed(42)
random_indices = np.random.permutation(len(bikeids))
X_random = n_trips_clusters[random_indices]
y_random = n_maints[random_indices]
X_train, X_test, y_train, y_test = train_test_split(X_random, y_random, test_size=0.2, random_state=42)

estimator = LinearRegression()
estimator.fit(X_train, y_train)
print estimator.score(X_test, y_test)

#parameters = {'n_neighbors': range(10, 20, 1)}
#estimator = GridSearchCV(KNeighborsRegressor(), parameters, cv=5)
estimator = KNeighborsRegressor(n_neighbors=99)
estimator.fit(X_train, y_train)
print estimator.score(X_test, y_test)
#print estimator.best_params_

In [ ]:
bikes_df = pd.DataFrame(
    np.hstack((
            bikeids.reshape(-1,1), 
            n_maints.reshape(-1,1),  
            n_trips.reshape(-1,1), 
            n_trips_months, 
            n_trips_clusters, 
        )), 
    columns=['bikeid', 'n_maints', 'n_trips',
             'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'SEP', 'AUG', 'OCT', 'NOV', 'DEC', 
             'CL1', 'CL2', 'CL3', 'CL4', 'CL5', 'CL6', 'CL7', 'CL8', 'CL9']
).set_index('bikeid')
#pickle.dump(bikes_df, open('bikes_df.pickle', 'w'))

In [ ]:
bikes_df = pickle.load(open('bikes_df.pickle', 'r'))

In [ ]:
bikes_df.head()

In [ ]:
np.random.seed(42)
random_indices = np.random.permutation(len(bikeids))
# range(2, 13) for JAN to NOV, range(14, 22) for CL1 to CL8
X_random = bikes_df[range(1,13)+range(14,22)].values[random_indices]
y_random = bikes_df.n_maints.values[random_indices]
X_train, X_test, y_train, y_test = train_test_split(X_random, y_random, test_size=0.2, random_state=42)

#estimator = KNeighborsRegressor(n_neighbors=45)
estimator = LinearRegression()
estimator.fit(X_train, y_train)
print estimator.score(X_test, y_test)
plt.plot(estimator.coef_)